In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=16000
sample_rate=16000
quantization_channels=256
#dilations=[2**i for i in range(8)]*20
#"residualDim=32
dilations=[2**i for i in range(9)]*1
residualDim=256
skipDim=512
filterSize=3
pad = np.sum(dilations)
lossrecord=[]
pad

511

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels,forX=False):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())

In [7]:
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)
xtrain,xval,xtest=mu_law_encode(xtrain,forX=True),mu_law_encode(xval,forX=True),mu_law_encode(xtest,forX=True)

In [8]:
'''xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd'''

'xmean,xstd = xtrain.mean(),xtrain.std()\nxtrain=(xtrain-xmean)/xstd\nxval=(xval-xmean)/xstd\nxtest=(xtest-xmean)/xstd'

In [9]:
#xtrain,ytrain=xtrain[:xtest.shape[0]],ytrain[:xtest.shape[0]]
#xval,yval=xval[:xtest.shape[0]],yval[:xtest.shape[0]]
xtrain=np.pad(xtrain, (pad, pad), 'constant')
xval=np.pad(xval, (pad, pad), 'constant')
xtest=np.pad(xtest, (pad, pad), 'constant')
yval=np.pad(yval, (pad, pad), 'constant')
ytrain=np.pad(ytrain, (pad, pad), 'constant')

In [10]:
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[-sampleSize:],yval[-sampleSize:]
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[:sampleSize],yval[:sampleSize]
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [11]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.tanhconvs = nn.ModuleList()
        self.sigmoidconvs = nn.ModuleList()
        self.skipconvs = nn.ModuleList()
        self.denseconvs = nn.ModuleList()
        for i, d in enumerate(dilations):
            self.tanhconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d))
            self.sigmoidconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d))
            self.skipconvs.append(nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1))
            self.denseconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1))
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        finallen = x.shape[-1]-2*pad
        x = self.causal(x)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()[:,:,dilation:-dilation]
            x1 = self.tanh(self.tanhconvs[i](x))
            x2 = self.sigmoid(self.sigmoidconvs[i](x))
            x = x1*x2
            cutlen = (x.shape[-1] - finallen)//2
            if(i == 0):skip_connections= (self.skipconvs[i](x)).narrow(2,int(cutlen),int(finallen))
            else :skip_connections += (self.skipconvs[i](x)).narrow(2,int(cutlen),int(finallen))
            x = self.denseconvs[i](x)
            x += xinput
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net()#.cuda()
criterion = nn.CrossEntropyLoss()#.cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(),weight_decay=1e-5)

In [16]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
    print(correct,'accurate')
    print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        listofpred = []
        for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
            output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)

        listofpred=[]
        for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
            output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    #idx = np.arange(xtrain.shape[-1] - 2 * sampleSize,1000)
    #176000
    idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,16000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        #cur=model.tanhconvs[0].weight.clone()
        output = model(data)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #assert((model.tanhconvs[0].weight == cur).cpu().numpy().all())
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            with open("./lossRecord/newmodellossfile.txt", "w") as f:
                for s in lossrecord:
                    f.write(str(s) +"\n")
            print('write finish')
            state={'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()}
            torch.save(state, 'newModifiedModel')
    test()
    val()

In [ ]:
for epoch in range(100000):
    train(epoch)
    test()

Train Epoch: 0 [0/235 (0%)] Loss:5.545600: , (0.520 sec/step)
write finish
Train Epoch: 0 [1/235 (0%)] Loss:5.502888: , (0.518 sec/step)
Train Epoch: 0 [2/235 (1%)] Loss:5.472961: , (0.518 sec/step)
Train Epoch: 0 [3/235 (1%)] Loss:5.322281: , (0.517 sec/step)
Train Epoch: 0 [4/235 (2%)] Loss:5.316733: , (0.516 sec/step)
Train Epoch: 0 [5/235 (2%)] Loss:5.044542: , (0.517 sec/step)
Train Epoch: 0 [6/235 (3%)] Loss:5.550271: , (0.516 sec/step)
Train Epoch: 0 [7/235 (3%)] Loss:5.203669: , (0.516 sec/step)
Train Epoch: 0 [8/235 (3%)] Loss:5.158384: , (0.516 sec/step)
Train Epoch: 0 [9/235 (4%)] Loss:5.140691: , (0.516 sec/step)
Train Epoch: 0 [10/235 (4%)] Loss:5.187211: , (0.516 sec/step)
Train Epoch: 0 [11/235 (5%)] Loss:5.118204: , (0.517 sec/step)
Train Epoch: 0 [12/235 (5%)] Loss:5.154421: , (0.516 sec/step)
Train Epoch: 0 [13/235 (6%)] Loss:5.432991: , (0.517 sec/step)
Train Epoch: 0 [14/235 (6%)] Loss:5.122524: , (0.516 sec/step)
Train Epoch: 0 [15/235 (6%)] Loss:5.266912: , (0.517

Train Epoch: 0 [128/235 (54%)] Loss:4.874162: , (0.522 sec/step)
Train Epoch: 0 [129/235 (55%)] Loss:4.927829: , (0.522 sec/step)
Train Epoch: 0 [130/235 (55%)] Loss:4.787798: , (0.522 sec/step)
Train Epoch: 0 [131/235 (56%)] Loss:4.530011: , (0.521 sec/step)
Train Epoch: 0 [132/235 (56%)] Loss:5.081553: , (0.521 sec/step)
Train Epoch: 0 [133/235 (57%)] Loss:4.508558: , (0.521 sec/step)
Train Epoch: 0 [134/235 (57%)] Loss:4.725989: , (0.521 sec/step)
Train Epoch: 0 [135/235 (57%)] Loss:4.655681: , (0.522 sec/step)
Train Epoch: 0 [136/235 (58%)] Loss:5.051129: , (0.521 sec/step)
Train Epoch: 0 [137/235 (58%)] Loss:5.296474: , (0.521 sec/step)
Train Epoch: 0 [138/235 (59%)] Loss:4.868423: , (0.521 sec/step)
Train Epoch: 0 [139/235 (59%)] Loss:4.870853: , (0.521 sec/step)
Train Epoch: 0 [140/235 (60%)] Loss:4.758810: , (0.520 sec/step)
Train Epoch: 0 [141/235 (60%)] Loss:4.607596: , (0.521 sec/step)
Train Epoch: 0 [142/235 (60%)] Loss:4.624784: , (0.521 sec/step)
Train Epoch: 0 [143/235 (

Train Epoch: 1 [19/235 (8%)] Loss:4.599872: , (0.523 sec/step)
Train Epoch: 1 [20/235 (9%)] Loss:4.809652: , (0.523 sec/step)
Train Epoch: 1 [21/235 (9%)] Loss:4.724790: , (0.524 sec/step)
Train Epoch: 1 [22/235 (9%)] Loss:4.845641: , (0.523 sec/step)
Train Epoch: 1 [23/235 (10%)] Loss:4.626373: , (0.524 sec/step)
Train Epoch: 1 [24/235 (10%)] Loss:4.782375: , (0.524 sec/step)
Train Epoch: 1 [25/235 (11%)] Loss:4.432680: , (0.524 sec/step)
Train Epoch: 1 [26/235 (11%)] Loss:4.765490: , (0.523 sec/step)
Train Epoch: 1 [27/235 (11%)] Loss:4.667933: , (0.523 sec/step)
Train Epoch: 1 [28/235 (12%)] Loss:4.345472: , (0.524 sec/step)
Train Epoch: 1 [29/235 (12%)] Loss:4.845573: , (0.524 sec/step)
Train Epoch: 1 [30/235 (13%)] Loss:4.749982: , (0.523 sec/step)
Train Epoch: 1 [31/235 (13%)] Loss:4.880745: , (0.523 sec/step)
Train Epoch: 1 [32/235 (14%)] Loss:4.418137: , (0.523 sec/step)
Train Epoch: 1 [33/235 (14%)] Loss:4.391097: , (0.524 sec/step)
Train Epoch: 1 [34/235 (14%)] Loss:4.568924:

Train Epoch: 1 [147/235 (63%)] Loss:4.799040: , (0.523 sec/step)
Train Epoch: 1 [148/235 (63%)] Loss:4.507712: , (0.523 sec/step)
Train Epoch: 1 [149/235 (63%)] Loss:4.727119: , (0.524 sec/step)
Train Epoch: 1 [150/235 (64%)] Loss:4.826294: , (0.524 sec/step)
Train Epoch: 1 [151/235 (64%)] Loss:4.914197: , (0.523 sec/step)
Train Epoch: 1 [152/235 (65%)] Loss:4.456666: , (0.523 sec/step)
Train Epoch: 1 [153/235 (65%)] Loss:4.724656: , (0.523 sec/step)
Train Epoch: 1 [154/235 (66%)] Loss:4.719204: , (0.523 sec/step)
Train Epoch: 1 [155/235 (66%)] Loss:4.527683: , (0.524 sec/step)
Train Epoch: 1 [156/235 (66%)] Loss:4.361449: , (0.524 sec/step)
Train Epoch: 1 [157/235 (67%)] Loss:4.352621: , (0.523 sec/step)
Train Epoch: 1 [158/235 (67%)] Loss:4.623940: , (0.524 sec/step)
Train Epoch: 1 [159/235 (68%)] Loss:4.425817: , (0.523 sec/step)
Train Epoch: 1 [160/235 (68%)] Loss:4.109634: , (0.523 sec/step)
Train Epoch: 1 [161/235 (69%)] Loss:4.544815: , (0.523 sec/step)
Train Epoch: 1 [162/235 (

Train Epoch: 2 [38/235 (16%)] Loss:4.198977: , (0.523 sec/step)
Train Epoch: 2 [39/235 (17%)] Loss:4.941766: , (0.523 sec/step)
Train Epoch: 2 [40/235 (17%)] Loss:4.724562: , (0.523 sec/step)
Train Epoch: 2 [41/235 (17%)] Loss:4.212036: , (0.523 sec/step)
Train Epoch: 2 [42/235 (18%)] Loss:4.811594: , (0.522 sec/step)
Train Epoch: 2 [43/235 (18%)] Loss:4.774730: , (0.523 sec/step)
Train Epoch: 2 [44/235 (19%)] Loss:4.573010: , (0.523 sec/step)
Train Epoch: 2 [45/235 (19%)] Loss:4.792631: , (0.523 sec/step)
Train Epoch: 2 [46/235 (20%)] Loss:4.370935: , (0.523 sec/step)
Train Epoch: 2 [47/235 (20%)] Loss:4.458915: , (0.524 sec/step)
Train Epoch: 2 [48/235 (20%)] Loss:4.345961: , (0.523 sec/step)
Train Epoch: 2 [49/235 (21%)] Loss:4.766860: , (0.524 sec/step)
Train Epoch: 2 [50/235 (21%)] Loss:4.717874: , (0.524 sec/step)
Train Epoch: 2 [51/235 (22%)] Loss:4.259240: , (0.523 sec/step)
Train Epoch: 2 [52/235 (22%)] Loss:4.420121: , (0.523 sec/step)
Train Epoch: 2 [53/235 (23%)] Loss:4.705

Train Epoch: 2 [165/235 (70%)] Loss:4.732305: , (0.523 sec/step)
Train Epoch: 2 [166/235 (71%)] Loss:4.316310: , (0.523 sec/step)
Train Epoch: 2 [167/235 (71%)] Loss:4.082152: , (0.523 sec/step)
Train Epoch: 2 [168/235 (71%)] Loss:4.156278: , (0.522 sec/step)
Train Epoch: 2 [169/235 (72%)] Loss:4.305772: , (0.523 sec/step)
Train Epoch: 2 [170/235 (72%)] Loss:4.283553: , (0.522 sec/step)
Train Epoch: 2 [171/235 (73%)] Loss:4.424940: , (0.523 sec/step)
Train Epoch: 2 [172/235 (73%)] Loss:4.593466: , (0.522 sec/step)
Train Epoch: 2 [173/235 (74%)] Loss:4.407276: , (0.523 sec/step)
Train Epoch: 2 [174/235 (74%)] Loss:4.463192: , (0.523 sec/step)
Train Epoch: 2 [175/235 (74%)] Loss:4.432986: , (0.523 sec/step)
Train Epoch: 2 [176/235 (75%)] Loss:4.211300: , (0.523 sec/step)
Train Epoch: 2 [177/235 (75%)] Loss:4.469188: , (0.523 sec/step)
Train Epoch: 2 [178/235 (76%)] Loss:4.575199: , (0.522 sec/step)
Train Epoch: 2 [179/235 (76%)] Loss:4.295825: , (0.523 sec/step)
Train Epoch: 2 [180/235 (

In [ ]:
#torch.save(model, 'loss2.5~3.5step10h_repeat5*2**9resu32sample50000')